# Data Engineering#

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
stations = pd.read_csv('Downloads\hawaii_stations.csv')
measures = pd.read_csv('Downloads\hawaii_measurements.csv')

In [3]:
measures.head()

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/6/2010,NaN,73


In [4]:
stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [5]:
stations

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [6]:
measures.fillna(0.00).head()

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/6/2010,0.00,73


In [7]:
measures = measures.fillna(0.00)

In [8]:
measures.head()

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/6/2010,0.00,73


In [9]:
measures.to_csv('clean_measurements.csv', index = False)
stations.to_csv('clean_stations.csv', index = False)

# Database Engineering#

In [10]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pymysql
pymysql.install_as_MySQLdb()

In [11]:
measures = pd.read_csv('clean_measurements.csv')
stations = pd.read_csv('clean_stations.csv')

In [12]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [13]:
conn = engine.connect()

In [14]:
Base = declarative_base()

In [15]:
class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(500))
    date = Column(String(500))
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Station(Base):
    __tablename__ = "stations"
    id = Column(Integer, primary_key = True)
    station = Column(String(500))
    name = Column(String(500))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [16]:
Base.metadata.create_all(engine)

In [17]:
measurements_data = measures.to_dict(orient='records')
measurements_data[0]

{'date': '1/1/2010', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [18]:
stations_data = stations.to_dict(orient='records')
stations_data[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [19]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [20]:
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [21]:
conn.execute(measurements_table.insert(), measurements_data)
conn.execute(stations_table.insert(), stations_data)

In [22]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '1/1/2010', 0.08, 65),
 (2, 'USC00519397', '1/2/2010', 0.0, 63),
 (3, 'USC00519397', '1/3/2010', 0.0, 74),
 (4, 'USC00519397', '1/4/2010', 0.0, 76),
 (5, 'USC00519397', '1/6/2010', None, 73)]

In [23]:
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]